In [1]:
from network import *
from data_utils import *
import hyperparams as hp

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

(3, 3)
(3, 3, 1)


In [2]:



class Graph:
    def __init__(self):
        self.graph = tf.Graph()

        with self.graph.as_default():
            self.x = tf.placeholder(tf.float32, [None, hp.timestep, 1], name='X')
            self.y = tf.placeholder(tf.float32, [None, hp.timestep, 1], name='Y')
            if hp.use_mulaw:
                label = mu_law_encode(self.y)
            else:
                label = self.y

            output = network(self.x, use_mulaw=hp.use_mulaw)

            if hp.use_mulaw:
                self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=label))
            else:
                self.loss = tf.reduce_mean(tf.abs(output - label))

            self.train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(self.loss)

            tf.summary.scalar("loss", self.loss)
            self.merged = tf.summary.merge_all()





In [3]:
def main():
    mixture = np.load('mixtures.npy')
    vocals = np.load('vocals.npy')

    num_batch = len(mixture) // hp.batch_size

    g = Graph()

    with g.graph.as_default():
        # config = tf.ConfigProto(allow_soft_placement=True)
        # config.gpu_options.allocator_type = 'BFC'
        # config.gpu_options.per_process_gpu_memory_fraction = 0.80
        # config.gpu_options.allow_growth = True

        saver = tf.train.Saver()
        config = tf.ConfigProto(log_device_placement=False)
        config.gpu_options.allow_growth=True
        with tf.Session(config=config) as sess:
            init = tf.global_variables_initializer()
            sess.run(init)
            for epoch in range(hp.num_epochs):

                mixture, vocals = dataset_shuffling(mixture, vocals)
                for i in range(num_batch):
                    batch_X, batch_Y = get_batch(mixture, vocals, i, hp.batch_size)
                    sess.run(g.train_op, feed_dict={g.x:batch_X, g.y:batch_Y})
                    
                    if i % 100 == 0:
                        print ("step %d, CEloss:%.4f" %(i,sess.run(g.loss, feed_dict={g.x:batch_X, g.y:batch_Y})))
                        saver.save(sess, hp.save_dir+"/model_%d.ckpt" % (epoch*num_batch + i))
if __name__ == '__main__':
    main()

step 0, CEloss:5.5451
step 100, CEloss:5.4199
step 200, CEloss:5.4069
step 300, CEloss:5.4606
step 400, CEloss:5.4648
step 500, CEloss:5.3003
step 600, CEloss:5.1486
step 0, CEloss:5.0631
step 100, CEloss:5.1750
step 200, CEloss:4.6630
step 300, CEloss:5.1749
step 400, CEloss:5.2861
step 500, CEloss:5.2800
step 600, CEloss:4.4998
step 0, CEloss:4.9673
step 100, CEloss:5.2402
step 200, CEloss:4.8528
step 300, CEloss:4.9784
step 400, CEloss:4.7279
step 500, CEloss:4.5135
step 600, CEloss:4.9976
step 0, CEloss:4.6076
step 100, CEloss:5.0825
step 200, CEloss:4.6838
step 300, CEloss:4.4015
step 400, CEloss:4.9556
step 500, CEloss:3.8656
step 600, CEloss:4.3603
step 0, CEloss:4.7017
step 100, CEloss:5.0740
step 200, CEloss:4.4884
step 300, CEloss:4.3380
step 400, CEloss:4.6254
step 500, CEloss:4.8325
step 600, CEloss:4.8355
step 0, CEloss:4.6322
step 100, CEloss:4.2886
step 200, CEloss:4.8775
step 300, CEloss:4.4251
step 400, CEloss:4.9479
step 500, CEloss:4.5694
step 600, CEloss:4.7003
step

step 300, CEloss:4.6829
step 400, CEloss:4.8072
step 500, CEloss:3.5398
step 600, CEloss:4.0275
step 0, CEloss:3.9234
step 100, CEloss:3.8431
step 200, CEloss:4.4794
step 300, CEloss:3.7478
step 400, CEloss:5.4548
step 500, CEloss:4.7180
step 600, CEloss:4.1765
step 0, CEloss:3.7755
step 100, CEloss:4.4591
step 200, CEloss:3.6355
step 300, CEloss:4.4899
step 400, CEloss:4.5145
step 500, CEloss:3.7662
step 600, CEloss:3.7125
step 0, CEloss:4.3199
step 100, CEloss:4.1965
step 200, CEloss:3.7807
step 300, CEloss:4.2215
step 400, CEloss:3.8498
step 500, CEloss:4.7171
step 600, CEloss:3.5677
step 0, CEloss:4.1671
step 100, CEloss:3.5673
step 200, CEloss:4.4017
step 300, CEloss:4.3022
step 400, CEloss:4.5074
step 500, CEloss:3.7564
step 600, CEloss:4.3624
step 0, CEloss:4.1782
step 100, CEloss:3.7516
step 200, CEloss:4.1121
step 300, CEloss:3.5454
step 400, CEloss:4.2806
step 500, CEloss:4.5851
step 600, CEloss:4.0863
step 0, CEloss:4.3665
step 100, CEloss:4.1247
step 200, CEloss:3.5915
step

step 600, CEloss:3.3868
step 0, CEloss:4.6922
step 100, CEloss:3.6847
step 200, CEloss:4.2069
step 300, CEloss:3.2014
step 400, CEloss:4.5567
step 500, CEloss:4.3736
step 600, CEloss:4.4247
